# import

In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
color = sns.color_palette()

# aggregate

In [211]:
#aggregate: spending加總

def aggr_rfm(df):
    grouped = df.groupby(
        [  
          'user_main'
           
        ]
    ).agg({
           'spending':'sum'
           ,'quantity':'sum'
           ,'id':'nunique'
           ,'date':['max','min']
    })
    df = grouped.reset_index()
    #reset_index()可將group by結果變成新dataframe
    #df.columns =['user_main', 'product_category', 'spending', 'quantity','id', 'date_max', 'date_min'] 可分保修
    df.columns =['user_main', 'spending', 'quantity','id', 'date_max', 'date_min'] 
    return df

# create frequency tag

In [212]:
def rfm_f_tag(df):
    
    df['freq_tag']=df2['id'].apply(lambda x: '1' if x ==1  else '2')
    
    return df

# create monetary tag

In [213]:
def rfm_m_tag(df):

    df['money_tag']=df['spending'].apply(
        lambda x: '3' if x >=2000   
        else '2' if x>=1000  
        else '1')
    
    return df

# create recency tag

In [214]:
def rfm_r_tag(df):
    
    df['recency'] = (df['recency']/np.timedelta64(1, 'D')).astype(int)
    df['recency_tag']=df['recency'].apply(
        lambda x: '1' if x >=360  
        else '2' if x>=180 
        else '3'if x>=90
        else '4')
    return df

# create member tag

In [1]:
def member_tags(df,count_until_date):
      # count_until_date =datetime.datetime.strptime(count_until_date, '%Y-%m-%d')
    #新客 
       if (df['date_min'].year ==count_until_date.year and df['date_min'].month==count_until_date.month) :
        return '新客'
    #忠誠客
       if (df['id'] >=2 and df['recency'] <=90):
        return '忠誠客'
    #活躍客
       if df['recency'] <= 90 :
        return '活躍客'
    #沉睡客
       if (df['recency'] >90 and df['recency'] <=360):
        return '沉睡客'
    #流失客
       if (df['recency'] >360):
        return '流失客'
    #其他  
       return 'Other'

# member tag count

In [216]:
def member_tag_count(df):
    member_tag_count=df.groupby(['member_tags','date']).agg({'user_main':'nunique'})
    member_tag_count.reset_index(inplace=True)
    return member_tag_count

# last day of month 

In [246]:
def last_day_of_month(date):
    
    date= datetime.datetime.strptime(date, '%Y-%m-%d')
    if date.month == 12:
        return date.replace(day=31)
    
    return datetime.datetime.strftime(date.replace(month=date.month+1, day=1) - datetime.timedelta(days=1),'%Y-%m-%d')

# new return tag

In [ ]:
def new_return_tags(df):
      # count_until_date =datetime.datetime.strptime(count_until_date, '%Y-%m-%d')
    #新客 
       if (df['date_min'].year ==df['date'].year and df['date_min'].month==df['date'].month) :
        return '新客'
    #回購客  
       return '回購客'

In [247]:
last_day_of_month('2018-03-01')

datetime.datetime(2018, 3, 31, 0, 0)

# main : calculate tags for each month

In [306]:
def main(count_until_date,date_back_month): 
    
    #讀進最新資料    
    data = pd.read_excel('aimee_sale_08.xlsx', sheetname='Sheet 1')
    
    #轉換時間格式
    data['date'] =pd.to_datetime(data['date'])    
    count_until_date= datetime.datetime.strptime(count_until_date, '%Y-%m-%d')
    year = count_until_date.year
    month = count_until_date.month
    
    #create a empty dataframe
    df3 = pd.DataFrame()

    for i in range(0,date_back_month):

        this_date =datetime.date(year,month-i,1)
        
        #每月最後一天
        last_date = last_day_of_month(datetime.datetime.strftime(this_date,'%Y-%m-%d'))
       
        #資料篩選:該月最後一天
        row = data.loc[data['date'] <= last_date].copy()  

        #新增單價: quantity x price
        row['spending'] = row['quantity']*row['price']
        df2 = aggr_rfm(row)

        #recency: 距離今天最後一次消費
        df2['recency'] =last_date - pd.to_datetime(df2['date_max'])
        
        # member_tags
        df2 = rfm_r_tag(df2)
        df2 = rfm_f_tag(df2)
        df2 = rfm_m_tag(df2)
        df2['rfm_tag'] =df2['recency_tag']+df2['freq_tag']+df2['money_tag']
        df2['member_tags'] = df2.apply (lambda row: member_tags(row, last_date),axis=1)
        df2['date']= last_date

        df3 = df3.append(member_tag_count(df2))
    
    print(df3)
    return df3

In [307]:
df = main('2018-07-31',4)

  member_tags       date  user_main
0         忠誠客 2018-07-31        100
1          新客 2018-07-31        201
2         沉睡客 2018-07-31       1620
3         活躍客 2018-07-31        325
4         流失客 2018-07-31        423
0         忠誠客 2018-06-30         74
1          新客 2018-06-30        208
2         沉睡客 2018-06-30       1564
3         活躍客 2018-06-30        259
4         流失客 2018-06-30        363
0         忠誠客 2018-05-31         75
1          新客 2018-05-31        150
2         沉睡客 2018-05-31       1690
3         活躍客 2018-05-31        227
4         流失客 2018-05-31        118
0         忠誠客 2018-04-30         48
1          新客 2018-04-30        126
2         沉睡客 2018-04-30       1642
3         活躍客 2018-04-30        259
4         流失客 2018-04-30         35


In [308]:
#df
df.to_csv("monthly_rmf_data.csv",index=False)

In [90]:

df = pd.read_excel('aimee_sale_08.xlsx', sheetname='Sheet 1')
df['month'] = df['date'].astype(str).str[0:7]

grouped = df.groupby(
    [ 
        'month'

    ]
).agg({
      
       'id':'nunique'
       ,'user_main':'nunique'
})

grouped.reset_index()

,month,id,user_main
0,2017-03,56,26
1,2017-04,17,15
2,2017-05,87,86
3,2017-06,248,246
4,2017-07,75,72
5,2017-08,149,146
6,2017-09,118,116
7,2017-10,232,231
8,2017-11,308,290
9,2017-12,348,316
